In [1]:
import ollama
import base64
import os
import json

In [2]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Literal

# --- Pydantic Schema Definitions ---

# Schema for a single anomaly detected
class Anomaly(BaseModel):
    modality: Literal['image', 'audio', 'sensor', 'overall'] = Field(description="The source of the anomaly.")
    description: str = Field(description="A brief description of the detected anomaly.")
    severity: Literal['Low', 'Medium', 'High', 'Critical'] = Field(description="The severity level of the anomaly.")

# Schema for the complete comparison response
class ComparisonReport(BaseModel):
    triage_Level: str = Field(description="The model generates the level of triage")
    triage_action: str = Field(description="The model generates the action to undertake")
    reasoning_summary: str = Field(description="The model generates the summary reason")

In [45]:
SYSTEM_PROMPT = ""

with open("system_prompt.txt", "r") as f:
    SYSTEM_PROMPT = f.read()

SYSTEM_PROMPT

'You are the ALZHEIMER\'S TRIAGE MONITORING AGENT (ATMA). Your sole purpose is to analyze multimodal inputs (image, voice transcript, and sensor data) to determine the immediacy of a life-threatening crisis for a dementia or Alzheimer\'s patient.\nYour response MUST be a final, decisive action classified into one of three Triage Levels. Do not engage in conversational chat or explain the diagnosis unless explicitly asked.\n-- TRIAGE PROTOCOL --\nINPUT ANALYSIS: You will receive the following:\nIMAGE: A visual assessment (e.g., patient location, posture, visible injury).\nVOICE TRANSCRIPT: A transcription of the patient or a nearby sound (e.g., patient vocalizations, distress, environmental sounds).\nSENSOR DATA: A short text summary of objective vitals or environmental data (e.g., Heart Rate, Temperature, Door Status).\n\nCRITICAL REASONING: Use the following rules, prioritizing the most severe evidence:\n\nCRISIS (Level 1): Trigger if any of the following combinations are present:\nEv

In [39]:

# --- 1. Configuration ---
# BASELINE SET
# B_IMAGE_PATH = "../data/baseline/Gemini_Generated_Image_f5zwnnf5zwnnf5zw.png"
B_IMAGE_PATH = "./compressed_data/Gemini_Generated_Image_f5zwnnf5zwnnf5zw_compressed.jpg"
B_AUDIO_PATH = "../data/baseline/-28U1_qW0sU_out.wav"
B_SENSOR_PATH = "../data/baseline/sensor_data.json" 


# Anomaly set
# A_IMAGE_PATH = "../data/min_anomaly/Gemini_Generated_Image_wnooknwnooknwnoo.png"
A_IMAGE_PATH = "./compressed_data/Gemini_Generated_Image_wnooknwnooknwnoo_compressed.jpg"
A_AUDIO_PATH = "../data/min_anomaly/-8Ezw9946g8_out.wav"
A_SENSOR_PATH = "../data/min_anomaly/sensor_data.json" 

# AA_IMAGE_PATH = "../data/max_anomaly/Gemini_Generated_Image_gubff8gubff8gubf.png"
AA_IMAGE_PATH = "./compressed_data/Gemini_Generated_Image_gubff8gubff8gubf_compressed.jpg"
AA_AUDIO_PATH = "../data/max_anomaly/-4pUrlMafww_out.wav"
AA_SENSOR_PATH = "../data/max_anomaly/sensor_data.json" 



# Choose your deployed omnimodal model
MODEL_NAME = "gemma3:12b"
MODEL_NAME = "gemma3:4b"


In [18]:
# --- Helper Function: File Encoding ---
def file_to_base64(file_path: str) -> str:
    # ... (function defined previously - encodes file to base64 string) ...
    # This function should be included in your final script
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Error: File not found at {file_path}")
    print(f"Encoding file: {file_path}...")
    with open(file_path, "rb") as f:
        base64_string = base64.b64encode(f.read()).decode("utf-8")
        print(f"File {os.path.basename(file_path)} encoded. Length: {len(base64_string)}")
        return base64_string

def load_sensor_data(file_path: str) -> str:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Error: File not found at {filepath}")
    print(f"Encoding file: {file_path}...")
    with open(file_path, "r") as f:
        return json.loads(f.read())


In [46]:
messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "system",
            "content": f"""
                Given is the baseline image and audio.  
                Given is the sensor data {json.dumps(load_sensor_data(B_SENSOR_PATH))}.
                Understand the base line and use it as a reference for comparison.
                """,
            "images": [file_to_base64(B_IMAGE_PATH)],  # Only the single Base64 string here
            "audio": [file_to_base64(B_AUDIO_PATH)]
        },
        {
            "role": "system",
            "content": f"""
                Given is the real time image and audio. 
                Given is the sensor data {json.dumps(load_sensor_data(AA_SENSOR_PATH))}.
                Compare with the baseline previously provided and generate the appropriate response as mentioned.
            """,
            "images": [file_to_base64(AA_IMAGE_PATH)],  # Only the single Base64 string here
            "audio": [file_to_base64(AA_AUDIO_PATH)]
        },
    ]


print(f"\nSending simple request to Ollama using model: {MODEL_NAME}...")
response = ollama.chat(
    model=MODEL_NAME,
    messages=messages,
    options={'temperature': 0.7, "num_threads": 10}
)

# 4. Print the model's response
print("\n--- Model Response ---")
print(response["message"]["content"])
print("----------------------")

Encoding file: ../data/baseline/sensor_data.json...
Encoding file: ./compressed_data/Gemini_Generated_Image_f5zwnnf5zwnnf5zw_compressed.jpg...
File Gemini_Generated_Image_f5zwnnf5zwnnf5zw_compressed.jpg encoded. Length: 152140
Encoding file: ../data/baseline/-28U1_qW0sU_out.wav...
File -28U1_qW0sU_out.wav encoded. Length: 2352104
Encoding file: ../data/max_anomaly/sensor_data.json...
Encoding file: ./compressed_data/Gemini_Generated_Image_gubff8gubff8gubf_compressed.jpg...
File Gemini_Generated_Image_gubff8gubff8gubf_compressed.jpg encoded. Length: 156968
Encoding file: ../data/max_anomaly/-4pUrlMafww_out.wav...
File -4pUrlMafww_out.wav encoded. Length: 1163192

Sending simple request to Ollama using model: gemma3:4b...

--- Model Response ---
```json
{
"triage_tevel": "Level 1",
"triage_action": "EMERGENCY",
"reasoning_summary": "The real-time image shows the patient lying on the floor with an elevated heart rate of 125 BPM and low oxygen saturation (88%). This combination of evidence

In [13]:
import requests
import json

def send_ollama_request(prompt_content):
    """Sends a prompt to the local Ollama API and prints the response."""
    
    # The default local address for the Ollama service
    url = "http://localhost:11434/api/chat"
    
    # Data payload for the API request
    payload = {
        "model": "gemma3:12b",  # Make sure you have this model pulled (ollama pull llama3)
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "stream": False  # Set to False to get the full response at once
    }
    
    # Send the POST request
    try:
        response = requests.post(url, data=json.dumps(payload))
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        
        # Parse the JSON response
        result = response.json()
        
        # Extract the assistant's message content
        assistant_message = result['message']['content']
        print(f"Model Response:\n{assistant_message}")
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        print("Please ensure Ollama is running and the model (llama3) is installed.")

# --- Run the function ---
send_ollama_request("write a python code for fibonacci series uptil 100, please respond in 50 words or less. NO MORE")


Model Response:
```python
a, b = 0, 1
while a <= 100:
  print(a, end=" ")
  a, b = b, a + b
```
This code generates the Fibonacci sequence up to 100 using iterative approach, printing each number separated by a space.


In [14]:
from PIL import Image
import os
import subprocess
from typing import Optional

def compress_media_file(input_path: str, output_dir: str = "./compressed_data", quality: int = 80) -> Optional[str]:
    """
    Compresses an image, audio, or video file to reduce size.
    Returns the path to the compressed file, or None if compression fails.

    :param input_path: Path to the original media file.
    :param output_dir: Directory to save the compressed file.
    :param quality: Compression quality (1-100, typically 80 is good).
    :return: Path to the compressed file.
    """
    if not os.path.exists(input_path):
        print(f"Error: Input file not found at {input_path}")
        return None

    os.makedirs(output_dir, exist_ok=True)
    file_name, file_ext = os.path.splitext(os.path.basename(input_path))
    
    # 1. Image Compression (Highest Impact: Convert PNG/BMP to JPEG)
    if file_ext.lower() in ['.png', '.bmp', '.jpeg', '.jpg']:
        output_path = os.path.join(output_dir, f"{file_name}_compressed.jpg")
        try:
            img = Image.open(input_path)
            # Force conversion to JPEG format for maximum compression (lossy)
            if img.mode == 'RGBA':
                img = img.convert('RGB')
            img.save(output_path, 'jpeg', quality=quality)
            print(f"Compressed image saved to: {output_path}")
            return output_path
        except Exception as e:
            print(f"Error compressing image {input_path}: {e}")
            return None

    # 2. Audio/Video Compression (Requires FFmpeg)
    # This uses a conceptual framework requiring the FFmpeg tool installed on the system PATH.
    elif file_ext.lower() in ['.mp3', '.wav', '.mp4', '.mov']:
        output_ext = '.mp3' if file_ext.lower() in ['.mp3', '.wav'] else '.mp4'
        output_path = os.path.join(output_dir, f"{file_name}_compressed{output_ext}")
        
        # FFmpeg command for re-encoding audio/video to lower bitrate
        if output_ext == '.mp3':
            # Example: Convert to 64kbps MP3 (very small)
            cmd = ['ffmpeg', '-i', input_path, '-b:a', '64k', output_path, '-y']
        else: # .mp4 video
            # Example: Re-encode video to crf 28 (higher compression, lower quality)
            cmd = ['ffmpeg', '-i', input_path, '-vcodec', 'libx264', '-crf', '28', '-preset', 'veryfast', output_path, '-y']
        
        try:
            subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(f"Compressed media saved to: {output_path}")
            return output_path
        except FileNotFoundError:
            print("Error: FFmpeg not found. Please ensure it is installed and in your system PATH.")
            return None
        except subprocess.CalledProcessError as e:
            print(f"Error compressing media {input_path}: {e.stderr.decode()}")
            return None

    else:
        print(f"Format {file_ext} not supported for internal compression. Returning original path.")
        return input_path

In [15]:
compress_media_file(B_IMAGE_PATH)
compress_media_file(A_IMAGE_PATH)

Compressed image saved to: ./compressed_data/Gemini_Generated_Image_f5zwnnf5zwnnf5zw_compressed.jpg
Compressed image saved to: ./compressed_data/Gemini_Generated_Image_wnooknwnooknwnoo_compressed.jpg


'./compressed_data/Gemini_Generated_Image_wnooknwnooknwnoo_compressed.jpg'

In [36]:
compress_media_file(AA_IMAGE_PATH)

Compressed image saved to: ./compressed_data/Gemini_Generated_Image_gubff8gubff8gubf_compressed.jpg


'./compressed_data/Gemini_Generated_Image_gubff8gubff8gubf_compressed.jpg'